In [1]:
import pandas as pd
from datetime import datetime
import sqlite3

In [12]:
import sqlite3

# 连接数据库
conn = sqlite3.connect('test.db')
cursor = conn.cursor()

# 创建 users 表
cursor.execute('''
CREATE TABLE IF NOT EXISTS users (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    age INTEGER NOT NULL,
    sex INTEGER NOT NULL,
    height REAL NOT NULL,
    weight REAL NOT NULL,
    exercise INTEGER NOT NULL,
    email TEXT NOT NULL UNIQUE,
    password TEXT NOT NULL
)
''')

# 创建 health_status 表
cursor.execute('''
CREATE TABLE IF NOT EXISTS health_status (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_id INTEGER NOT NULL,
    K REAL,
    P REAL,
    Glucose_random REAL,
    Glucose_AC REAL,
    LDL_C REAL,
    eGPR REAL,
    datetime DATETIME,
    FOREIGN KEY (user_id) REFERENCES users(id)
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS daily_nutrition (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_id INTEGER NOT NULL,
    datetime DATETIME,
    calories REAL,
    carbohydrate REAL,
    protein REAL,
    fat REAL,
    K REAL,
    P REAL,
    Na REAL,
    Ca REAL,
    VitA REAL,
    VitD REAL,
    VitE REAL,
    VitK REAL,
    FOREIGN KEY (user_id) REFERENCES users(id),
    UNIQUE (user_id, datetime)
)
''')

# 提交更改并关闭连接
conn.commit()
conn.close()


In [11]:
import sqlite3

def drop_table():
    conn = sqlite3.connect('test.db')
    cursor = conn.cursor()

    # Drop the daily_nutrition table
    cursor.execute("DROP TABLE IF EXISTS users;")
    cursor.execute("DROP TABLE IF EXISTS daily_nutrition;")
    cursor.execute("DROP TABLE IF EXISTS health_status;")
    conn.commit()
    conn.close()

# Call the function to drop the table
drop_table()


In [13]:
import bcrypt

def hash_password(password):
    return bcrypt.hashpw(password.encode('utf-8'), bcrypt.gensalt()).decode('utf-8')

def add_user(name, email, age, password, height, weight, sex, exercise):
    conn = sqlite3.connect('test.db')
    cursor = conn.cursor()
    hashed_password = hash_password(password)
    if sex == 'female':
        sex_tag = int(1)
    else:
        sex_tag = int(0)
    height = float(height)
    weight = float(weight)
    cursor.execute("INSERT INTO users (name, email, age, password, height, weight, sex, exercise) VALUES (?, ?, ?, ? ,? ,?, ?, ?)", (name, email, age, hashed_password, height, weight, sex_tag, exercise))
    conn.commit()
    conn.close()

def verify_password(stored_password, provided_password):
    return bcrypt.checkpw(provided_password.encode('utf-8'), stored_password.encode('utf-8'))

def login_user(email, password):
    correct = False
    conn = sqlite3.connect('test.db')
    cursor = conn.cursor()
    cursor.execute("SELECT password FROM users WHERE email = ?", (email,))
    result = cursor.fetchone()
    if result and verify_password(result[0], password):
        print("Login successful")
        correct = True
    else:
        print("Invalid credentials")
    conn.commit()
    conn.close()
    return correct

def find_user_id(email):
    conn = sqlite3.connect('test.db')
    cursor = conn.cursor()
    cursor.execute("SELECT id FROM users WHERE email = ?", (email, ))
    conn.commit()
    return cursor.fetchone()  # This will return the first match or None if not found

def get_user_health_data(user_id):
    conn = sqlite3.connect('test.db')
    cursor = conn.cursor()
    cursor.execute('''
    SELECT users.name, users.email, users.age, health_status.K, health_status.P, health_status.Glucose_random, health_status.Glucose_AC, health_status.LDL_C, health_status.eGPR, health_status.datetime
    FROM users
    JOIN health_status ON users.id = health_status.user_id
    WHERE users.id = ?
    ''', (user_id,))
    conn.commit()
    return cursor.fetchall()


def find_user_data(email, password):
    correct = login_user(email, password)
    if not correct:
        return
    id = find_user_id(email)
    user_health_data = get_user_health_data(id[0])
    for data in user_health_data:
        print(data)

def add_health_status(email, password, K, P, Glucose_random, Glucose_AC, LDL_C, eGPR, datetime):
    correct = login_user(email, password)
    if not correct:
        return
    id = find_user_id(email)
    conn = sqlite3.connect('test.db')
    cursor = conn.cursor()
    cursor.execute("INSERT INTO health_status (user_id, K, P, Glucose_random, Glucose_AC, LDL_C, eGPR, datetime) VALUES (?, ?, ?, ?, ?, ?, ?, ?)",
                   (id[0], K, P, Glucose_random, Glucose_AC, LDL_C, eGPR, datetime))
    conn.commit()
    conn.close()

In [14]:
add_user("alice", "alice@example.com", 30, "securepassword123", 165, 45, 'female', 2)

In [15]:
from datetime import date
today = date.today()
today_str = today.strftime('%Y-%m-%d')
add_health_status("alice@example.com", "securepassword123", 6, 6, 200, 100, 100, 50, today_str)

Login successful


In [7]:
find_user_data("alice@example.com", "securepassword123")

Login successful
('alice', 'alice@example.com', 30, 6.0, 6.0, 200.0, 100.0, 100.0, '2024-11-4')


In [ ]:
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('test.db')
cursor = conn.cursor()

# Delete a specific row
cursor.execute("DELETE FROM daily_nutrition WHERE id = ?", (66,))

# Commit the changes and close the connection
conn.commit()
conn.close()
